In [6]:
!pip install kfp --upgrade -q

In [7]:
import kfp
import kfp.components as comp
from kfp import dsl

import glob
import pandas as pd
import tarfile
import urllib.request

In [10]:
def merge_csv(file_path: comp.InputPath('Tarball'),
              output_csv: comp.OutputPath('CSV')):
    import glob
    import pandas as pd
    import tarfile

    tarfile.open(name=file_path, mode="r|gz").extractall('data')
    df = pd.concat(
        [pd.read_csv(csv_file, header=None)for csv_file in glob.glob('data/*.csv')])
    df.to_csv(output_csv, index=False, header=False)

In [11]:
create_step_merge_csv = kfp.components.create_component_from_func(
        func=merge_csv,
        output_component_file='component.yaml', # This is optional. It saves the component spec for future use.
        base_image='python:3.7',
        packages_to_install=['pandas==1.1.4'])

In [13]:
web_downloader_op = kfp.components.load_component_from_url(
            'https://raw.githubusercontent.com/kubeflow/pipelines/240543e483076ae718f82c6f280441daa2f041fd/components/web/Download/component.yaml')

In [19]:
# from kubernetes.client.models import V1Toleration, V1Affinity, V1NodeAffinity, V1NodeSelector, V1NodeSelectorTerm, V1NodeSelectorRequirement

@dsl.pipeline(
    name="base_pipeline",
    description="my base pipeline",
)
def my_pipeline(url):
    web_downloader_task = web_downloader_op(url=url)
    merge_csv_task = create_step_merge_csv(file=web_downloader_task.outputs['data'])

#     toleration = V1Toleration(
#         key="kubernetes.azure.com/scalesetpriority",
#         operator="Equal",
#         value="spot",
#         effect="NoSchedule",
#     )
#     web_downloader_task.add_toleration(toleration)
#     merge_csv_task.add_toleration(toleration)

#     aff = V1Affinity(node_affinity=V1NodeAffinity(
#         required_during_scheduling_ignored_during_execution=V1NodeSelector(
#             node_selector_terms=[V1NodeSelectorTerm(
#                 match_expressions=[V1NodeSelectorRequirement(
#                     key='kubernetes.azure.com/scalesetpriority',
#                     operator='In',
#                     values=['spot'])])]))
#     )
#     web_downloader_task.add_affinity(aff)
#     merge_csv_task.add_affinity(aff)

In [25]:
import os
os.environ['KF_PIPELINES_SA_TOKEN_PATH'] = "/var/run/secrets/kubernetes.io/serviceaccount/token"

In [26]:
client = kfp.Client()

In [27]:
client.create_run_from_pipeline_func(
    my_pipeline,
    arguments={
        'url': 'https://storage.googleapis.com/ml-pipeline-playground/iris-csv-files.tar.gz'
    })

ApiException: (500)
Reason: Internal Server Error
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json', 'Date': 'Thu, 27 Jan 2022 13:25:26 GMT', 'Transfer-Encoding': 'chunked'})
HTTP response body: {"error":"Internal error: [Unauthenticated: Request header error: there is no user identity header.: Request header error: there is no user identity header., Authentication failure: Unauthenticated: Review.Status.Authenticated is false: Failed to authenticate token review]\nFailed to authorize with API resource references\ngithub.com/kubeflow/pipelines/backend/src/common/util.Wrap\n\t/go/src/github.com/kubeflow/pipelines/backend/src/common/util/error.go:275\ngithub.com/kubeflow/pipelines/backend/src/apiserver/server.(*ExperimentServer).canAccessExperiment\n\t/go/src/github.com/kubeflow/pipelines/backend/src/apiserver/server/experiment_server.go:249\ngithub.com/kubeflow/pipelines/backend/src/apiserver/server.(*ExperimentServer).ListExperiment\n\t/go/src/github.com/kubeflo